# Practice Exercises for Single Table SQL

Given a dataset and a set of questions, form the queries to answer the questions. All questions in this particular exercise only require a query to a single table. 

⚠️ *Note:* Please display minimal number of attributes (columns) needed to make sense of your answer. Not too many, not too few.


## Setup
The SQLite database for these exercises has been setup for you. For the curious, you can look at this file for the DDL commands to setup the tables and insert data.  `db_illum_ddl_5_3.sql`

Here is the schema for the database. The arrows show foreign keys, which is just fyi for now. 

![worker_project schema](worker_project_schema.png) 

In [1]:
# Make sure to always run this cell when you open this notebook
# Start the Jupyter SQL engine, connecting to a SQLite database 
%reload_ext sql 
%sql sqlite:///worker_project.db

0. In the command line, inspect the sqlite database (worker_project.db) to see what the tables and columns actually are. Any differences you observe? (Also, copy past your output in the cell below)

In [ ]:
sqlite3 worker_project.db
SQLite version 3.38.2 2022-03-26 13:51:10
Enter ".help" for usage hints.
sqlite> .tables
Assign   Dept     Project  Worker 
sqlite> select * from Assign;
1001|101|30|
1001|103|20|5
1005|103|20|
1001|105|30|
1001|115|20|4
1019|110|20|5
1019|115|10|4
1025|110|10|
1030|110|10|
sqlite> select * from Dept
   ...> ;
Accounting|101
Research|110
sqlite> select * from Project;
1001|Jupiter|101|300000|01-Feb-2021|50
1005|Saturn|101|400000|01-Jun-2022|35
1019|Mercury|110|350000|15-Feb-2022|40
1025|Neptune|110|600000|01-Feb-2021|45
1030|Pluto|110|380000|15-Sep-2023|50
sqlite> select * from Worker;
101|Smith|Tom|Accounting|01-Feb-1970|06-Jun-1993 |50000
103|Jones|Mary|Accounting|15-Jun-1975|20-Sep-2005|48000
105|Burns|Jane|Accounting|21-Sep-1980|12-Jun-2015|39000
110|Burns|Michael|Research|05-Apr-1977|10-Sep-2020|70000
115|Chin|Amanda|Research|22-Sep-1980|19-Jun-2022|60000

1. Get the names of all workers in the Accounting department.

In [10]:
%%sql
Select (firstName || " " || lastName) as full_name from Worker where deptName == "Accounting"

 * sqlite:///worker_project.db
Done.


full_name
Tom Smith
Mary Jones
Jane Burns


2. Get the name of the employee in the Research department who has the lowest salary.

In [16]:
%%sql
Select (firstName || " " || lastName) as full_name from Worker where deptName == "Research"
Order by salary ASC limit 1

 * sqlite:///worker_project.db
Done.


full_name
Amanda Chin


3. Get the details of the project with the highest budget.

In [20]:
%%sql
Select * from Project
Order by budget DESC limit 1

 * sqlite:///worker_project.db
Done.


projNo,projName,projMgrId,budget,startDate,expectedDurationWeeks
1025,Neptune,110,600000,01-Feb-2021,45


## The following questions may require more than one query to answer. 
Show your work! Add as many SQL cells as needed for each question. Use comments to indicate how the output of each query is used to arrive at your answer.  

4. List out alphabetically the names of all workers working on Project 1001.

In [28]:
%%sql
Select (firstName || " " || lastName) as full_name from Worker, Assign where Worker.empId = Assign.empId and Assign.projNo = 1001
Order by full_name ASC

 * sqlite:///worker_project.db
Done.


full_name
Amanda Chin
Jane Burns
Mary Jones
Tom Smith


5. Get an alphabetical list of names and corresponding ratings for all workers on any project managed by Michael Burns.

In [40]:
%%sql
Select DISTINCT (firstName || " " || lastName) as full_name, rating from Worker, Project, Assign where Worker.empId = Assign.empId and Assign.projNo = (Select projNo from Project where projMgrId = 110)
Order by full_name ASC 

 * sqlite:///worker_project.db
Done.


full_name,rating
Amanda Chin,4
Michael Burns,5


6. For all projects starting after Feb 1, 2022, find the project number and names of all workers assigned to them. 

In [48]:
%%sql
--Select projNo, (firstName || " " || lastName) as full_name from Worker, Assign where projNo in (Select projNo from Project where startDate > "01-Feb-2021")
Select projNo from Project where startDate > "01-Feb-2021"

 * sqlite:///worker_project.db
Done.


projNo
1005
1019
1030


In [50]:
%%sql
Select empId from Assign where projNo in (Select projNo from Project where startDate > "01-Feb-2021")

 * sqlite:///worker_project.db
Done.


empId
103
110
115
110


In [52]:
%%sql 
Select (firstName || " " || lastName) as full_name from Worker where empId in (Select empId from Assign where projNo in (Select projNo from Project where startDate > "01-Feb-2021"))

 * sqlite:///worker_project.db
Done.


full_name
Mary Jones
Michael Burns
Amanda Chin


7. Find the names of employees who are not assigned as managers any project.

In [55]:
%%sql
Select DISTINCT projMgrId from Project

 * sqlite:///worker_project.db
Done.


projMgrId
101
110


In [56]:
%%sql 
Select (firstName || " " || lastName) as full_name from Worker where empId NOT IN (Select DISTINCT projMgrId from Project)

 * sqlite:///worker_project.db
Done.


full_name
Mary Jones
Jane Burns
Amanda Chin


8. Find details of any project with the string "urn" in the project name. 

In [57]:
%%sql
Select * from Project where projName LIKE "%urn%"

 * sqlite:///worker_project.db
Done.


projNo,projName,projMgrId,budget,startDate,expectedDurationWeeks
1005,Saturn,101,400000,01-Jun-2022,35


9. List out the projects from most to least efficient. (hint: the project with the lowest budget is not necessarily the one that uses time and human resources effectively.)

In [60]:
%%sql
Select empId, projNo from Assign

 * sqlite:///worker_project.db
Done.


empId,projNo
101,1001
103,1001
105,1001
115,1001
103,1005
110,1019
115,1019
110,1025
110,1030


In [64]:
%%sql
Select projName, budget/expectedDurationWeeks/(SELECT count() from Assign where projNo=p.projNo) as "project_efficiency" 
from Project p
Order by project_efficiency DESC

 * sqlite:///worker_project.db
Done.


projName,project_efficiency
Neptune,13333
Saturn,11428
Pluto,7600
Mercury,4375
Jupiter,1500


10. Explain your reasoning for your solution to #9 (use a Markdown cell), including who should get the award for Most Efficient Project Manager.

My query for #9 decides efficiency by dividing the budget of each project by the estimated duration in weeks and by the number of employees assigned to those projects, at least I THINK the count() portion is counting the number of employees on each project. I chose to approach it this way because I thought the duration was important, but the number of employees on each project also vastly changes how efficient a project would be. Thus, dividing the total budget of each project by the estimated duration and the number of employees on the project should give an accurate show of how the budget is divided among the employees on the project over the duration. The higher the project_efficiency number is the more efficient the project is. The Most Efficient Project Manager award should go to Michael Burns because he has the most efficient project along with the 3rd and 4th most efficient projects.

In [70]:
%%sql

Select * from Project

 * sqlite:///worker_project.db
Done.


projNo,projName,projMgrId,budget,startDate,expectedDurationWeeks
1001,Jupiter,101,300000,01-Feb-2021,50
1005,Saturn,101,400000,01-Jun-2022,35
1019,Mercury,110,350000,15-Feb-2022,40
1025,Neptune,110,600000,01-Feb-2021,45
1030,Pluto,110,380000,15-Sep-2023,50


11. Bonus!!  A Query that you can't easily answer with just a set of queries: Get a list of project numbers with name and start dates, of all the projects that have the same start date as another. 
*you don't have to answer this, but think about it!*


## Submission:

Commit and push your changes to this document by the deadline, and share a link to the repository in the Moodle assignment.  

I worked with Molly Perez on most of the queries, and received help from Prof Ramsey and Chesky. 
Dr. Jones said that I did not have to go back and change the queries using joins since I had already completed them, but I made sure to complete the rest without joins. Also, the queries that ask for alphabetized results are alphabetized by first name.